##### Import libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import os
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

##### Import dataset

In [2]:
df=pd.read_csv('medical_questions.csv', encoding='utf-8')

In [3]:
df.shape

(29435, 2)

For medical question classification, it requires high-quality datasets to train a deep-learning approach in a supervised way. Currently, there is no public dataset for Azerbaijani medical classification, and the datasets of other fields are not applicable to the medical QA system. To solve this problem, I scraped a m.tibb.az website using Python where 29435 questions in 19 medical branch have been asked by users and answered by medical experts. I will also provide dataset which can be used in Azerbaijani medical QA and related fields.

In [4]:
# Check the df

df.head()

,doctor_name,questions
0,Psixoloq \nElnur Rüstəmov,\n\t\t\t\t\t\t\t\t\t\tSalam Elnur bey.Men urey...
1,Psixoloq \nElnur Rüstəmov,\n\t\t\t\t\t\t\t\t\t\tSalam Doktor menim sizin...
2,Psixoloq \nElnur Rüstəmov,\n\t\t\t\t\t\t\t\t\t\tSalam.Beynim tormuzlanib...
3,Psixoloq \nElnur Rüstəmov,\n\t\t\t\t\t\t\t\t\t\t salam Elnur hekim menim...
4,Psixoloq \nElnur Rüstəmov,\n\t\t\t\t\t\t\t\t\t\tQİzim Diqqetsizdi ve oz ...


In [5]:
df.isnull().sum()

doctor_name    0
questions      0
dtype: int64

In [6]:
# unique categories

print(df['doctor_name'].unique())

['Psixoloq \nElnur Rüstəmov' 'Psixoterapevt \nRəna Tağıyeva'
 'Ümumi cərrah \nRafail Abdullayev' 'Plastik cərrah \nXəyal Aşurəliyev'
 'Dermato veneroloq \nŞahin Bədəlov' 'Hepatoloq \nNicat Tağıyev'
 'Cərrah-proktoloq \nElgün Səmədov' 'Nevropatoloq \nFərhad Əhmədov'
 'Ginekoloq \nNaibə Həsənova' 'LOR həkimi \nHafiz Mehdiyev'
 'Allerqoloq \nFəxri Hüseynov' 'Kardioloq \nRəna Səmədova'
 'Endokrinoloq \nMənsurə Şıxəliyeva' 'Qastroenteroloq \nSəbirə Səfərova'
 'Endoskopist \nUmud Mahmudov' 'Uroloq \nMahir Şirinov'
 'Oftalmoloq \nKönül Feyziyeva' 'Pediatr \nCəmilə Hüseynova'
 'Proktoloq \nDilarə Həsənova']


In [7]:
df['questions']=df['questions'].str.strip('\n').str.strip('\t')

In [8]:
df['profession']=df['doctor_name'].map(lambda row: row.split('\n')[0])

In [9]:
df.drop('doctor_name', axis=1, inplace=True)

In [10]:
df.head()

,questions,profession
0,Salam Elnur bey.Men ureyimde nese bosluq hiss ...,Psixoloq
1,Salam Doktor menim sizin komeyinize ehtiyacİç ...,Psixoloq
2,Salam.Beynim tormuzlanib ele bil.Oxdugum kitab...,Psixoloq
3,salam Elnur hekim menim 17 yasim var real he...,Psixoloq
4,Qİzim Diqqetsizdi ve oz guvensiZdi 5 yaslidi e...,Psixoloq


In [11]:
professions=list(df['profession'].unique())

print(professions)

['Psixoloq ', 'Psixoterapevt ', 'Ümumi cərrah ', 'Plastik cərrah ', 'Dermato veneroloq ', 'Hepatoloq ', 'Cərrah-proktoloq ', 'Nevropatoloq ', 'Ginekoloq ', 'LOR həkimi ', 'Allerqoloq ', 'Kardioloq ', 'Endokrinoloq ', 'Qastroenteroloq ', 'Endoskopist ', 'Uroloq ', 'Oftalmoloq ', 'Pediatr ', 'Proktoloq ']


In [12]:
labels={professions[index]: index for index in range(len(professions)) }

print(labels)

{'Psixoloq ': 0, 'Psixoterapevt ': 1, 'Ümumi cərrah ': 2, 'Plastik cərrah ': 3, 'Dermato veneroloq ': 4, 'Hepatoloq ': 5, 'Cərrah-proktoloq ': 6, 'Nevropatoloq ': 7, 'Ginekoloq ': 8, 'LOR həkimi ': 9, 'Allerqoloq ': 10, 'Kardioloq ': 11, 'Endokrinoloq ': 12, 'Qastroenteroloq ': 13, 'Endoskopist ': 14, 'Uroloq ': 15, 'Oftalmoloq ': 16, 'Pediatr ': 17, 'Proktoloq ': 18}


In [13]:
# convert string labels to integers

df['labels'] = df['profession'].map(labels)

In [14]:
df.head()

,questions,profession,labels
0,Salam Elnur bey.Men ureyimde nese bosluq hiss ...,Psixoloq,0
1,Salam Doktor menim sizin komeyinize ehtiyacİç ...,Psixoloq,0
2,Salam.Beynim tormuzlanib ele bil.Oxdugum kitab...,Psixoloq,0
3,salam Elnur hekim menim 17 yasim var real he...,Psixoloq,0
4,Qİzim Diqqetsizdi ve oz guvensiZdi 5 yaslidi e...,Psixoloq,0


In [15]:
df=df.drop('profession', axis=1)

In [16]:
# Let's create a train and test set

train, test = train_test_split(df, test_size=0.1, random_state=42)

In [17]:
# Lets define the model with the parameters (important here is the number of labels and nr of epochs)

model = ClassificationModel('bert', 'bert-base-multilingual-uncased', num_labels=df['labels'].nunique(), 
                            use_cuda=False, args={'reprocess_input_data': True, 
                                  'overwrite_output_dir': True, 'num_train_epochs': 3, 
                                   "train_batch_size": 64, 'learning_rate': 5e-5,
                                    'gradient_accumulation_steps': 1,
                                    'batch_size': 8, 'max_grad_norm': 5,
                                    'patience': 2, 'warmup_proportion': 1})

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# Now lets fine tune bert with the train set

model.train_model(train)

  0%|                                                 | 0/26491 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environ

Epochs 0/3. Running Loss:    2.0628:  19%|▉    | 79/414 [08:41<36:40,  6.57s/it]


Epochs 0/3. Running Loss:    1.9653:  31%|█▏  | 128/414 [13:59<31:23,  6.58s/it]


Epochs 0/3. Running Loss:    1.4485:  43%|█▋  | 176/414 [19:17<26:01,  6.56s/it]


Epochs 0/3. Running Loss:    1.6719:  54%|██▏ | 225/414 [24:36<20:40,  6.56s/it]


...

Epochs 2/3. Running Loss:    0.6837:  76%|███ | 316/414 [34:50<10:51,  6.65s/it]


Epochs 2/3. Running Loss:    0.9030:  88%|███▌| 365/414 [40:11<05:24,  6.61s/it]


Epochs 2/3. Running Loss:    1.1889: 100%|███▉| 413/414 [45:31<00:06,  6.61s/it]


Epoch 3 of 3: 100%|███████████████████████████| 3/3 [2:16:33<00:00, 2731.11s/it]


(1242, 1.212741823276076)

In [19]:
# Let's evaluate this finetuned model with the test set
result, model_outputs, wrong_predictions = model.eval_model(test)

  0%|                                                  | 0/2944 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environ

In [20]:
predictions = model_outputs.argmax(axis=1)

In [21]:
predictions[0:10]

array([ 4,  8,  4,  1,  1, 15, 16,  1,  4,  8])

In [22]:
actuals = test.labels.values

In [23]:
actuals[0:10]

array([10,  8,  4,  1,  1,  2, 16,  1,  4,  8])

In [24]:
# Now lets see the accuracy one the test set

print(accuracy_score(actuals, predictions))

0.6735733695652174


In [25]:
def show_model_result(model_result):
    
    result = model_result[0][0]
    
    for label in labels.keys():
        
        if labels[label]==result:
            
            return label
        
    return

In [26]:
# Lets predict the text of sample_text:
sample_text='Üzümdə səpgi var.'

model_result=model.predict([sample_text])

  0%|                                                     | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 13.02it/s]


In [27]:
print(show_model_result(model_result))

Allerqoloq 
